### 1. 基本

#### 1.1初始化参数
    - static_url_path
    - static_folder，默认：'static'
    - template_folder， 默认：'templates'

#### 1.2 配置参数
    - app.config.from_pyfile('config.cfg')
    - app.config.from_object()

#### 1.3 在视图用读取配置参数
    - app.config.get()
    - current_app.config.get()

#### 1.4 app.run 的参数
    - app.run(host='0.0.0.0')

### 2. 路由

#### 2.1 查看所有路由

In [ ]:
# 查看所有路由
app.url_map

#### 2.2 限制访问方式

In [ ]:
# 限制访问方式 GET/POST
@app.route('/sample', methods=['GET', 'POST'])

#### 2.3 使用 url_for 进行反解析

In [ ]:
# 使用 url_for 进行反解析
url_for('函数名')

#### 2.4  动态路由

In [ ]:
# 动态路由
@app.route('/user/<int:id>')
def hello_itcast(id):
  return 'hello itcast %d' %id

#### 2.5 转换器

##### 2.5.1 内置转换器

  | 类型   | 说明                       |
  | ------ | -------------------------- |
  | string | 默认处理方式               |
  | int    | 接受整数                   |
  | float  | 接受浮点数                 |
  | path   | 和默认的相似，但也接受斜线 |

##### 2.5.2 自定义转换器

In [ ]:
# 自定义转换器
from flask import Flask
from werkzeug.routing import BaseConverter

class Regex_url(BaseConverter):
  def __init__(self,url_map,*args):
      super(Regex_url,self).__init__(url_map)
      self.regex = args[0]

app = Flask(__name__)
app.url_map.converters['re'] = Regex_url

@app.route('/user/<re("[a-z]{3}"):id>')
def hello_itcast(id):
  return 'hello %s' %id

### 3. request 获取请求参数

#### 3.1 导入依赖
- from flask import request

#### 3.2 常用属性

|  属性   |              说明              |     类型      |
| :-----: | :----------------------------: | :-----------: |
|  data   | 记录请求的数据，并转换为字符串 |       *       |
|  form   |       记录请求中表单数据       |   MultiDict   |
|  args   |       记录请求中查询参数       |   MultiDict   |
| cookies |      记录请求中cookie信息      |     Dict      |
| headers |        记录请求中报文头        | EnvironHeader |
| method  |    记录请求中使用的HTTP方法    |   GET/POST    |
|   url   |       记录请求中URL地址        |    string     |
|  files  |       记录请求中上传文件       |       *       |

#### 3.3 上传文件 示例

In [ ]:
# 上传文件 示例
from flask import request
from werkzeug import secure_filename

@app.route('/upload', methods=['GET', 'POST'])
def upload_file():
  if request.method == 'POST':
      f = request.files['the_file']
      f.save('/var/www/uploads/' + secure_filename(f.filename))

### 4. 请求上下文(request context)
- request和session都属于请求上下文对象。

### 5. 应用上下文(application context)
- current_app和g都属于应用上下文对象。
  - current_app:表示当前运行程序文件的程序实例。
  - g:处理请求时，用于临时存储的对象，每次请求都会重设这个变量。

### 6. 上下文管理器 with

In [ ]:
# 示例
with open('1.txt', 'wb') as f:
    f.write('hello work')

### 7. cookie

#### 7.1 设置

In [ ]:
# 设置
resp = make_response('success')
resp.set_cookie('cast','python')
# 设置过期时间
resp.set_cookie('cast1','python1', max_age=3600)

#### 7.2 获取

In [ ]:
# 获取
request.cookies.get('cast')

#### 7.3 删除

In [ ]:
# 删除
resp = make_response('success')
resp.delete_cookie('cast', 'python')

### 8. session

#### 8.1 导入依赖
- from flask import session

#### 8.2 设置

In [ ]:
# flask 必须设置secret_key
app.config['SECRET_KEY'] = 'asfasdgfsdg'
session['key'] = 'value'

#### 8.3 获取

In [ ]:
#获取
session.get('key')

### 9. 返回响应数据

#### 9.1 元组
  - 元组必须是 (response, status, headers) 的形式，且至少包含一个元素。 status 值会覆盖状态代码， headers 可以是一个列表或字典，作为额外的消息标头值

In [ ]:
# 示例
resp = make_response()
resp.headers[“sample”] = “value”
resp.status = “404 not found”

#### 9.2 JSON 模块
  - json.dumps(字典)  将python的字典转换为json字符串
  - json.loads(字符串) 将字符串转换为python中的字典

In [ ]:
# 示例
data = {
    'name': 'python',
    'age': 24
}
jsonify(data)
jsonify(city='zs')

### 10. 异常处理

#### 10.1 abort函数

##### 10.1.1 导入依赖
- from flask import abort

##### 10.1.2 使用

In [ ]:
# 传递状态码，必须是标准的http状态码
abort(403)

# 传递响应体信息
resp = Response('err msg')
abort(resp)

#### 10.2 自定以异常

In [ ]:
@app.errhandle(404)
def handle_404_error(err):
  ###这个函数返回前端用户看到的最终结果  ###
  return '出现404异常，{}'.format(err)

### 11. 请求钩子

#### 11.1 flask 支持以下四种请求钩子：

- before_first_request：在处理第一个请求前运行。 
  - @app.before_first_request 
- before_request：在每次请求前运行。 
- after_request(response)：如果没有未处理的异常抛出，在每次请求后运行。 
- teardown_request(response)：在每次请求后运行，即使有未处理的异常抛出。工作在非调试模式下

### 12. 蓝图 Blueprint
- 蓝图：用于实现单个应用的视图、模板、静态文件的集合

#### 12.1 使用

In [ ]:
# 创建蓝图对象
#Blueprint必须指定两个参数，admin表示蓝图的名称，__name__表示蓝图所在模块
admin = Blueprint('admin',__name__)


# 注册蓝图路由
@admin.route('/')
def admin_index():
  return 'admin_index'

# 在程序实例中注册该蓝图
app.register_blueprint(admin,url_prefix='/admin')

### 13. 单元测试

#### 13.1 常用的断言方法
- assertEqual     如果两个值相等，则pass
- assertNotEqual  如果两个值不相等，则pass
- assertTrue      判断bool值为True，则pass
- assertFalse     判断bool值为False，则pass
- assertIsNone    不存在，则pass
- assertIsNotNone 存在，则pass

#### 13.2 单元测试的基本写法

In [ ]:
# 首先，定义一个类，继承自unittest.TestCase
import unittest
class TestClass(unitest.TestCase):
    pass



# 其次，在测试类中，定义两个测试方法
import unittest
class TestClass(unittest.TestCase):

    #该方法会首先执行，方法名为固定写法
    def setUp(self):
        pass

    #该方法会在测试代码执行完后执行，方法名为固定写法
    def tearDown(self):
        pass

    
# 最后，在测试类中，编写测试代码
import unittest
class TestClass(unittest.TestCase):

    #该方法会首先执行，相当于做测试前的准备工作
    def setUp(self):
        pass

    #该方法会在测试代码执行完后执行，相当于做测试后的扫尾工作
    def tearDown(self):
        pass
    #测试代码
    def test_app_exists(self):
        pass